# Assess Processed Outputs

Process results from RAP and see variant retention and read depth and allelic balance.

# imports, globals, and data

In [2]:
import pandas as pd
import numpy as np
DATA_OUT = "/mnt/speliotes-lab/Projects/UK_ATLAS/IndivProj/craut/UKB_500K_exome_QC/"
rap_output_files = "{}rap_runs/{}/".format(DATA_OUT,"{}")
GROUPS = ["group_{}".format(i) for i in range(1,10+1)]
from glob import glob

# Process Results

In [9]:
new_data_list = []
for dname in glob(rap_output_files[:-3]+"*"):
    if "." not in dname:
        for fname in glob(dname+"/*"):
            with open(fname, "r") as f:
                for line in f:
                    line = line.strip().split()
                    for snp in line[0].split(";"):
                        new_data_list.append({"snpid":snp,
                                              "rd_10":float(line[1]),
                                              "ab":float(line[2])})
new_data_df = pd.DataFrame(new_data_list)
print(new_data_df.shape)
new_data_df.head()

(3129424, 3)


,snpid,rd_10,ab
0,chr11_6410531_C_G,0.0,0.000000
1,chr11_6410534_A_G,0.0,0.000000
2,chr11_6410536_C_T,6.0,0.222222
3,chr11_6410540_C_T,7.0,0.181818
4,chr11_6410543_C_G,7.0,0.470588


In [10]:
# 10%_read_depth
indel_mask = np.array([bool(snp[-2] != "_" or snp[-4] != "_") for snp in new_data_df.snpid])
rd_mask = np.array([rd >= (10 if indel else 7) for indel,rd in zip(indel_mask,new_data_df.rd_10)])
ab_mask = np.array([ab >= (0.2 if indel else 0.15) for indel,ab in zip(indel_mask,new_data_df.ab)])
rd10_mask = rd_mask & ab_mask
print(np.sum(rd10_mask),len(rd10_mask),np.sum(rd10_mask)/len(rd10_mask))

2769156 3129424 0.8848772170214071
